In [18]:
from azure.ai.ml import automl, Input, MLClient
from azure.ai.ml.entities import ResourceConfiguration
from azure.identity import DefaultAzureCredential
from azure.ai.ml.constants import AssetTypes

# Suponemos que los datos ya han sido cargados y divididos, como en el código previo.
train_mltable_path = "./train_data/" 
valid_mltable_path = "./valid_data/" 

train_data_input = Input(type=AssetTypes.MLTABLE, path=train_mltable_path)
valid_data_input = Input(type=AssetTypes.MLTABLE, path=valid_mltable_path)

compute_name = "co2-cluster"
experiment_name = 'automl-co2-tuning'

# Configuramos el job de AutoML para regresión
automl_job = automl.regression(
    compute=compute_name,
    experiment_name=experiment_name,
    training_data=train_data_input,
    validation_data=valid_data_input,
    target_column_name="CO2 Emissions(g/km)",
)

automl_job.set_limits(
    timeout_minutes=30,
)

# Definir los algoritmos y hiperparámetros que queremos tunear
automl_job.set_training(
    allowed_training_algorithms=['LightGBM', 'ElasticNet'],
)

In [19]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(
    task='regression',
    primary_metric='normalized_root_mean_squared_error',
    experiment_timeout_minutes=30,
    training_data=train_dataset, # Asumiendo que ya has cargado tus datos en un objeto Dataset de Azure
    label_column_name='target', # El nombre de tu columna objetivo
    n_cross_validations=5,
    enable_early_stopping=True,
    featurization='auto',
    model_explainability=True,
    blocked_models=['KNN', 'LinearRegressor'], # Bloqueamos modelos que no queremos probar
    whitelist_models=['LightGBM', 'RandomForest'], # Lista de modelos a probar
    hyperparameter_sampling={
        'LightGBM': {
            'learning_rate': [0.01, 0.05, 0.1, 0.5],
            'num_leaves': [31, 50, 100, 150]
        },
        'RandomForest': {
            'n_estimators': [100, 200, 500],
            'max_depth': [10, 20, 30, None]
        }
    }
)


ModuleNotFoundError: No module named 'azureml.train'